In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the first dataset to get the column labels
first_df = pd.read_excel('Labeled_Data/scraped_articles_business_2_with_sentiment.xlsx')
columns = first_df.columns  # Extract column names

# Load all sheets and concatenate using the columns from the first dataset
sheets = ['Labeled_Data/scraped_articles_business_2_with_sentiment.xlsx', 'Labeled_Data/scraped_articles_business1_with_sentiment.xlsx'
          , 'Labeled_Data/scraped_articles_tech_with_sentiment.xlsx', 'Labeled_Data/scraped_articles_with_sentiment.xlsx']
dfs = [pd.read_excel(sheet, names=columns) for sheet in sheets]  # Use the same column names
df = pd.concat(dfs, ignore_index=True)
print(df.tail())

                                                  Title  \
1006  NZ will be ready to go from first ball: Daryl ...   
1007  India shouldn't send team for CWG: Ex-coach Vi...   
1008  Perth Scorchers sign pacer from Indonesia for ...   
1009  Netflix shuts down AAA game development studio...   
1010  Pep Guardiola comments on Kevin de Bruyne's re...   

                                                   Data   class sentiment  
1006  New Zealand all-rounder Daryl Mitchell said th...  sports  positive  
1007  Ex-India badminton coach Vimal Kumar criticise...  sports  negative  
1008  Perth Scorchers have signed Indonesia seamer N...  sports  positive  
1009  AAA game development studio, Team Blue by Netf...  sports  positive  
1010  Head coach Pep Guardiola spoke at the pre-matc...  sports  positive  


In [20]:
from sklearn.utils import resample
import pandas as pd

positive = df[df['sentiment'] == 'positive']
negative = df[df['sentiment'] == 'negative']
neutral = df[df['sentiment'] == 'neutral']

negative_upsampled = resample(negative, replace=True, n_samples=len(positive), random_state=42)
neutral_upsampled = resample(neutral, replace=True, n_samples=len(positive), random_state=42)

df_upsampled = pd.concat([positive, negative_upsampled, neutral_upsampled])

print(df_upsampled['sentiment'].value_counts())


sentiment
positive    629
negative    629
neutral     629
Name: count, dtype: int64


In [21]:
df_upsampled.tail(10)

,Title,Data,class,sentiment
57,Eni to sell 25% stake in biofuel unit to KKR,Italian group will use investment to help fund...,stock business,neutral
449,SpaceX wins $733 million launch contract from ...,Elon Musk-led SpaceX has won a $733 million co...,technology,neutral
483,Elon Musk donates $75 million to pro-Trump gro...,Billionaire Elon Musk donated around $75 milli...,technology,neutral
465,Netflix sees 35% QoQ jump in ads membership; Q...,Netflix has posted 35% quarter-on-quarter jump...,technology,neutral
668,Production at Tata's iPhone plant in Tamil Nad...,Tata Electronics has indefinitely suspended pr...,technology,neutral
889,No more Bazball: Rizwan teases Brook for playi...,Pakistan wicketkeeper Muhammad Rizwan teased E...,sports,neutral
112,Emerging markets are having a moment,US interest rate cuts spur reassessment of ass...,stock business,neutral
10,Politics is distorting economic data,Partisanship continues to pollute results of i...,stock business,neutral
423,Jellysmack laying off employees amid reorganis...,"Jellysmack, a SoftBank-backed creator-economy ...",technology,neutral
685,"Batted at 8 in IPL 2024 to give Jadeja, Dube c...",Discussing his decision of batting at eight in...,sports,neutral


In [22]:
df_upsampled['combined_text'] = df_upsampled['Title'] + ': ' + df_upsampled['Data']

In [23]:
df_upsampled.tail(10)

,Title,Data,class,sentiment,combined_text
57,Eni to sell 25% stake in biofuel unit to KKR,Italian group will use investment to help fund...,stock business,neutral,Eni to sell 25% stake in biofuel unit to KKR: ...
449,SpaceX wins $733 million launch contract from ...,Elon Musk-led SpaceX has won a $733 million co...,technology,neutral,SpaceX wins $733 million launch contract from ...
483,Elon Musk donates $75 million to pro-Trump gro...,Billionaire Elon Musk donated around $75 milli...,technology,neutral,Elon Musk donates $75 million to pro-Trump gro...
465,Netflix sees 35% QoQ jump in ads membership; Q...,Netflix has posted 35% quarter-on-quarter jump...,technology,neutral,Netflix sees 35% QoQ jump in ads membership; Q...
668,Production at Tata's iPhone plant in Tamil Nad...,Tata Electronics has indefinitely suspended pr...,technology,neutral,Production at Tata's iPhone plant in Tamil Nad...
889,No more Bazball: Rizwan teases Brook for playi...,Pakistan wicketkeeper Muhammad Rizwan teased E...,sports,neutral,No more Bazball: Rizwan teases Brook for playi...
112,Emerging markets are having a moment,US interest rate cuts spur reassessment of ass...,stock business,neutral,Emerging markets are having a moment: US inter...
10,Politics is distorting economic data,Partisanship continues to pollute results of i...,stock business,neutral,Politics is distorting economic data: Partisan...
423,Jellysmack laying off employees amid reorganis...,"Jellysmack, a SoftBank-backed creator-economy ...",technology,neutral,Jellysmack laying off employees amid reorganis...
685,"Batted at 8 in IPL 2024 to give Jadeja, Dube c...",Discussing his decision of batting at eight in...,sports,neutral,"Batted at 8 in IPL 2024 to give Jadeja, Dube c..."


In [24]:
! pip install transformers tensorflow datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/reetvikchatterjee/anaconda3/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [31]:
from datasets import Dataset

data = {
    'combined_text': df_upsampled['combined_text'],  # Use your combined text column
    'label': df_upsampled['class']  # Replace with your actual labels column
}

label_mapping = {
    'sports': 0,
    'technology': 1,
    'stock business': 2
    # Add more mappings if you have more classes
}

# Convert the labels
data['label'] = data['label'].map(label_mapping)


# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(pd.DataFrame(data))
dataset = dataset.train_test_split(test_size=0.2)

In [32]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [37]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [38]:
def tokenize_function(examples):
    return tokenizer(examples["combined_text"], padding="max_length", truncation=True)

# Apply tokenization
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format for TensorFlow
tokenized_train_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])


train_tf_dataset = tokenized_train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['label'],
    shuffle=True,
    batch_size=8
)

test_tf_dataset = tokenized_test_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['label'],
    shuffle=False,
    batch_size=8
)

Map:   0%|          | 0/1509 [00:00<?, ? examples/s]

Map:   0%|          | 0/378 [00:00<?, ? examples/s]

In [39]:
import tensorflow as tf

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.fit(train_tf_dataset, epochs=3, validation_data=test_tf_dataset)


Epoch 1/3
189/189 [==============================] - 656s 3s/step - loss: 0.3988 - accuracy: 0.8635 - val_loss: 0.1524 - val_accuracy: 0.9497
Epoch 2/3
189/189 [==============================] - 634s 3s/step - loss: 0.1223 - accuracy: 0.9622 - val_loss: 0.1240 - val_accuracy: 0.9603
Epoch 3/3
169/189 [=========================>....] - ETA: 1:03 - loss: 0.0740 - accuracy: 0.9793

In [40]:
loss, accuracy = model.evaluate(test_tf_dataset)
print(f"Test Accuracy: {accuracy}")


48/48 [==============================] - 44s 900ms/step - loss: 1.1134 - accuracy: 0.2407
Test Accuracy: 0.24074074625968933
